# DX-STREAM Tutorial 3 — Custom Model Integration
_Last updated: 2025-10-19 22:39_

### Goals
- Swap in your own `.dxnn` model
- Implement custom **preprocess**/**postprocess** libraries
- Wire everything via JSON configs and run

## 1) Prepare Your Model
1. Convert **ONNX → DXNN** using DX-COM (not covered in detail here).
2. Confirm tensor shapes with a parser (example):

```bash
parse_model -m your_model.dxnn
```
Record output names/shapes for your postprocess.

## 2) JSON Config Templates
Below are minimal examples. Adjust to your model specifics.

In [ ]:
pre_json = {
  "preprocess_id": 1,
  "resize_width": 640,
  "resize_height": 640,
  "keep_ratio": True,
  "library_file_path": "/usr/share/dx-stream/lib/libcustompreproc.so",
  "function_name": "CustomPreprocessFunc"
}
infer_json = {
  "preprocess_id": 1,
  "inference_id": 1,
  "model_path": "./models/your_model.dxnn",
  "secondary_mode": False
}
post_json = {
  "inference_id": 1,
  "library_file_path": "/usr/share/dx-stream/lib/libpostprocess_custom.so",
  "function_name": "Postprocess_Custom"
}
from pathlib import Path
cfg_dir = Path("configs"); cfg_dir.mkdir(exist_ok=True)
(Path("configs/preprocess.json")).write_text(__import__('json').dumps(pre_json, indent=2))
(Path("configs/infer.json")).write_text(__import__('json').dumps(infer_json, indent=2))
(Path("configs/postprocess.json")).write_text(__import__('json').dumps(post_json, indent=2))
print("Wrote configs to ./configs/")

## 3) Custom Preprocess (C++ Template)
Create a minimal preprocessor that writes into the provided `input_tensor` buffer. Do **not** re-allocate or free it.

In [ ]:
from pathlib import Path
src_dir = Path("custom_libs"); src_dir.mkdir(exist_ok=True)
(Path("custom_libs/preprocess.cpp")).write_text(r'''
#include <cstring>
#include <opencv2/opencv.hpp>
#include "dx_frame_meta.hpp"  // Provided by DX-STREAM headers
extern "C" void CustomPreprocessFunc(DXFrameMeta *frame_meta, DXObjectMeta *object_meta, void* input_tensor) {
    // Example: copy a resized BGR frame into input_tensor as float32 NHWC
    // NOTE: replace this with the correct layout/normalization for your model.
    if(!frame_meta || !frame_meta->_buf) return;
    // (Pseudo) fetch width/height from your config or tensor spec
    const int W = 640, H = 640, C = 3;
    cv::Mat src; // You would map frame_meta->_buf to cv::Mat using caps info
    // For a template, use a dummy image (black):
    cv::Mat resized(H, W, CV_8UC3, cv::Scalar(0,0,0));
    // Convert to float and normalize [0,1]
    cv::Mat f32; resized.convertTo(f32, CV_32FC3, 1.0/255.0);
    // Copy NHWC float32 into input_tensor
    std::memcpy(input_tensor, f32.ptr<float>(0), H*W*C*sizeof(float));
}
''')
print("Wrote custom_libs/preprocess.cpp")

## 4) Custom Postprocess (C++ Template)
Decode tensors to structured objects (bboxes, classes, scores) and attach to `DXFrameMeta`. For secondary-mode, modify existing `DXObjectMeta`.

In [ ]:
(Path("custom_libs/postprocess.cpp")).write_text(r'''
#include <vector>
#include "dx_post_meta.hpp"   // Provided by DX-STREAM headers
#include "dx_tensor.hpp"
extern "C" void Postprocess_Custom(std::vector<dxs::DXTensor> network_output,
                                   DXFrameMeta *frame_meta, DXObjectMeta *object_meta) {
    // Example: create one dummy detection
    if(!frame_meta) return;
    DXObjectMeta det{};
    det.rect.x = 0.1f; det.rect.y = 0.1f; det.rect.w = 0.5f; det.rect.h = 0.5f;
    det.score = 0.99f;
    det.class_id = 0;
    dx_attach_object_meta(frame_meta, &det);
}
''')
print("Wrote custom_libs/postprocess.cpp")

## 5) Build with Meson/Ninja (Template)
This assumes DX-STREAM headers & libs are installed system-wide (e.g., `/usr/local/include/dx_stream`).

In [ ]:
(Path("custom_libs/meson.build")).write_text(r'''
project('dx_stream_custom_libs', 'cpp', version : '1.0.0', default_options: ['cpp_std=c++17'])

dx_stream_dep = declare_dependency(
  include_directories : include_directories('/usr/local/include/dx_stream'),
  link_args : ['-L/usr/local/lib', '-lgstdxstream']
)

gst_dep = dependency('gstreamer-1.0', required : true)
opencv_dep = dependency('opencv4', required : true)

shared_library('custompreproc',
  sources: ['preprocess.cpp'],
  dependencies: [dx_stream_dep, gst_dep, opencv_dep],
  install: true,
  install_dir: '/usr/share/dx-stream/lib'
)

shared_library('postprocess_custom',
  sources: ['postprocess.cpp'],
  dependencies: [dx_stream_dep, gst_dep],
  install: true,
  install_dir: '/usr/share/dx-stream/lib'
)
''')
print("Wrote custom_libs/meson.build")

In [ ]:
# Build commands (run on your system, may require sudo for install paths)
# !sudo apt-get update && sudo apt-get install -y meson ninja-build
# !meson setup builddir custom_libs && ninja -C builddir && sudo ninja -C builddir install

## 6) Run a Standalone Pipeline (Template)
Example using a local MP4 and your custom configs:


In [ ]:
pipeline = r'''
filesrc location=./video.mp4 ! decodebin ! dxpreprocess config-file-path=./configs/preprocess.json ! dxinfer config-file-path=./configs/infer.json ! dxpostprocess config-file-path=./configs/postprocess.json ! dxosd width=1280 height=720 ! videoconvert ! autovideosink sync=false
'''.strip()
print(pipeline)
# To run from notebook (uncomment when ready):
# import subprocess, shlex
# subprocess.run(shlex.split("gst-launch-1.0 "+pipeline), check=False)

## 7) Add Multi-Object Tracking (Optional)
Insert `dxtracker` after `dxpostprocess` to get stable IDs. Example config:


In [ ]:
tracker_cfg = {
  "tracker_name": "OC_SORT",
  "params": {
    "det_thresh": 0.5,
    "max_age": 30,
    "min_hits": 3,
    "iou_threshold": 0.3
  }
}
Path("configs/tracker.json").write_text(__import__('json').dumps(tracker_cfg, indent=2))
print("Wrote configs/tracker.json")
print("Pipeline: add 'dxtracker config-file-path=./configs/tracker.json' before dxosd")

## 8) Tips & Best Practices
- Keep `preprocess-id` / `inference-id` consistent across Pre/Infer/Post.
- Use JSON files to make experiments reproducible.
- Ensure your postprocess matches **tensor layout** and **scales**.
- For remote sessions, prefer `autovideosink` or save to filesink during development.
- Add `dxrate throttle=true` after `dxinfer` if you need to reduce NPU load in low-FPS pipelines.